For this notebook, I am using
```python
!pip install numpy==1.26.1
!pip install tqdm==4.66.2

```

In [6]:
import numpy as np
from tqdm import tqdm
from collections import namedtuple
from typing import NamedTuple

In [7]:
class DifferentialEvolution():
    '''
    Implements Differential Evolution solver

    Parameters
    -----------
    func: callable
        The function to be minimized.
        It must in the form f(x, data) where x is the array with the parameters values to be assessed,
        Data is an immutable container comprised of the data.
    kmut: float
        It is the mutation constant. It needs  to be in the range (0, 1]
    kcross: float
        It is the crossover constant
    nparam: int
        It is the amount of parameters that are going to be fitter
    npop: int
        It is the population size.
    niter: int
        It is the number of iterations 
    data: flo
    '''
    
    def __init__(self, func, kmut, kcross, nparam, npop, niter, data = None, bounds = None, guess = None, seed = None):
        self.func = func
        self.kmut = kmut
        self.kcross = kcross
        self.nparam = nparam
        self.npop = npop
        self.niter = niter
        self.data = data
        self.bounds = bounds
        self.guess = guess
        np.random.seed(seed)
        
    def population_generator(self):
        '''
        Generate a random population n parameter x m population (size) from the uniform distribution.
        If there is a guess, it will be the first element of the population.
        If there is bounds, then the population parameters will be bounded by the values specified.
        Bounds should be a tuple comprised of a tuple where the first element is the lower bound and the second is the upper bound 
        '''
        pop = np.random.rand(self.nparam, self.npop)
        if self.bounds:
            pop = np.array([self.bounds[ii][0] + pop[ii] * (self.bounds[ii][1] - self.bounds[ii][0]) for ii in range(len(pop))])
        if self.guess:
            pop[:, 0] = np.array(self.guess)
        return pop.T

    def least_error_idx(self, pop):
        '''
        Identify the element from the population with the least error.
        The parameter func should have the following parameters: array to compute the error, list of array of the data,
        and list of sizes of each data. 
        Returns the index of the element with the least error and the least error
        '''
        error = np.zeros(self.npop)
        for idx, row in enumerate(pop):
            error[idx] = self.func(row, self.data)
        lst_idx = np.argmin(error)
        return lst_idx, error[lst_idx]

    def diff_evolution_solver(self):
        '''
        Computes the best fit via the differential evolution algorithm
        '''
        pop = self.population_generator()
        idx_list = np.arange(self.npop)
        lsterr_idx, lst_err = self.least_error_idx(pop)
        bestfit = pop[lsterr_idx]
        for iter in tqdm(range(self.niter)):  
            for i, ind in enumerate(pop):
                rng_idx = np.random.choice(idx_list, 2, replace = False)
                trial = bestfit + kmut * (pop[rng_idx[0]] - pop[rng_idx[1]])
                cross = np.concatenate((np.random.rand(3) <= self.kcross, np.array([True])))
                trial = np.where(cross, trial, ind)
                for j, param in enumerate(trial):
                    if not (param >= bounds[j][0] and param <= bounds[j][1]):
                         trial[j] = pop[:, j].min() + np.random.uniform() * (pop[:, j].max() - pop[:, j].min())
                trial_err = self.func(trial, self.data)
                if trial_err < self.func(ind, self.data):
                    pop[i] = trial
                    if trial_err < lst_err:
                        bestfit = trial
                        lst_err = trial_err
                        yield bestfit, lst_err

    @property
    def result(self):
        res, err  = zip(*tuple(self.diff_evolution_solver()))
        return res[-1]
        

In [8]:
def func_obj(r: float, cor1: float, cor2: float, n: np.ndarray, yraw: np.ndarray, stdraw: np.ndarray) -> float:
    y = 1/ (r * (1/np.square(3 - cor1) - 1/np.square(n - cor2)))
    err = np.square(yraw - y) / np.square(stdraw) #chi-square goodness of fit ((MSWD))
    return err

def compute_error(ind: np.ndarray, data_collection: tuple[NamedTuple]) -> float:
    '''
    Computes the error function, which is the chi-square goodness of fit
    This is the object function that we are trying to minimize.
    '''
    error = 0
    for data in data_collection:
        if data.name ==  'principal':
            error += np.sum(func_obj(
                ind[0],
                ind[1],
                ind[2],
                data.level,
                data.wavelength,
                data.uncertainty)
                           )
        elif data.name ==  'sharp':
            error += np.sum(func_obj(
                ind[0],
                ind[2],
                ind[1],
                data.level,
                data.wavelength,
                data.uncertainty)
                           )
        elif data.name ==  'diffuse':
            error += np.sum(func_obj(
                ind[0],
                ind[2],
                ind[3],
                data.level,
                data.wavelength,
                data.uncertainty)
                           )
        elif data.name ==  'balmer':
            error += np.sum(func_obj(
                ind[0],
                1,
                0,
                data.level,
                data.wavelength,
                data.uncertainty)
                           )
    return np.sqrt(error)

In [9]:
R=1.0e7
S=1.5
P=0.5
D=0.0

boundR = (1e6, 1e8)
boundS = (0.5, 2.0)
boundP = (0.3, 1.5)
boundD = (0.0, 0.1)

bounds = (boundR, boundS, boundP, boundD)
guess = (R, S, P, D)

In [10]:
principal = ((3, 5.86175E-7, 1e-8),
             (4, 3.3003E-7, 1e-8))
sharp = ((5, 6.1435E-7, 1e-8),
         (5, 6.1381E-7, 1e-8),
         (6, 5.1326E-7, 1e-8),   
         (6, 5.1284E-7, 1e-8),   
         (7, 4.7319E-7, 1e-8),   
         (7, 4.7294E-7, 1e-8),   
         (8, 4.5707E-7, 1e-8))
diffuse = ((4, 5.6722E-7, 1e-8),
          (4, 5.6661E-7, 1e-8),
          (5, 4.9618E-7, 1e-8),
          (5, 4.9576E-7, 1e-8),    
          (6, 4.6486E-7, 1e-8),
          (6, 4.6450E-7, 1e-8),
          (7, 4.4856E-7, 1e-8))
balmer = ((3, 6.5930E-7, 1e-9),
          (4, 4.8580E-7, 1e-9),
          (5, 4.3415e-7, 1e-9),
          (6, 4.1056e-7, 1e-9),
          (7, 3.9760e-7, 1e-9))

In [11]:
principal = np.array(principal, dtype = float)
sharp = np.array(sharp, dtype = float)
diffuse = np.array(diffuse, dtype = float)
balmer = np.array(balmer, dtype = float)

In [12]:
series = namedtuple('series', ('name', 
                               'level', 
                               'wavelength', 
                               'uncertainty'))
principal_data = series('principal', 
                       principal[:, 0], 
                       principal[:, 1], 
                       principal[:, 2])
sharp_data = series('sharp', 
                   sharp[:, 0], 
                   sharp[:, 1], 
                   sharp[:, 2])
diffuse_data = series('diffuse', 
                     diffuse[:, 0], 
                     diffuse[:, 1], 
                     diffuse[:, 2])
balmer_data = series('balmer', 
                    balmer[:, 0], 
                    balmer[:, 1], 
                    balmer[:, 2])

In [13]:
data_collection = (principal_data, sharp_data, diffuse_data, balmer_data)

In [14]:
niter = 10000
kmut = 0.2
kcross = 0.6

In [15]:
a = DifferentialEvolution(func = compute_error, 
                          kmut = kmut, 
                          kcross = kcross, 
                          nparam = 4, 
                          npop =10, 
                          niter = niter, 
                          data = data_collection, 
                          bounds = bounds, 
                          guess = guess,
                          seed = 0)

In [16]:
a.result

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:22<00:00, 448.35it/s]


array([1.09496219e+07, 1.45431074e+00, 9.03026874e-01, 4.87572644e-02])